<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# critic.py — модуль для оценки "здравости" сгенерированного текста с помощью PyTorch

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from typing import List, Tuple

################################################################################
# 🧠 1. Класс TorchCritic — модель для оценки осмысленности текста
################################################################################

class TorchCritic(nn.Module):
    """
    Простая модель-критик: Embedding → BiLSTM → Классификатор.
    Возвращает вероятность того, что текст "осмысленный".
    """

    def __init__(self, vocab_size: int, embed_dim: int = 64, hidden_dim: int = 128, coherence_threshold: float = 0.7):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)  # Преобразуем токены в векторы
        self.encoder = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.classifier = nn.Linear(2 * hidden_dim, 1)  # Предсказываем "осмысленность"
        self.coherence_threshold = coherence_threshold  # Порог, ниже которого отклоняется результат

    def forward(self, input_ids: torch.Tensor) -> torch.Tensor:
        """
        input_ids: (batch_size, seq_len) — батч токенов
        output: (batch_size,) — вероятность "осмысленности" для каждого примера
        """
        embedded = self.embedding(input_ids)                 # → (B, T, D)
        _, (hidden, _) = self.encoder(embedded)              # hidden: (2, B, H)
        concat_hidden = torch.cat((hidden[0], hidden[1]), dim=1)  # → (B, 2H)
        logits = self.classifier(concat_hidden)              # → (B, 1)
        probs = torch.sigmoid(logits)                        # → [0, 1]
        return probs.squeeze(1)                              # → (B,)

    def is_acceptable(self, input_ids: torch.Tensor) -> bool:
        """
        Возвращает True, если текст прошёл по порогу осмысленности.
        input_ids: (seq_len,) — одиночный пример
        """
        self.eval()
        with torch.no_grad():
            prob = self(input_ids.unsqueeze(0))  # → (1,)
            return prob.item() >= self.coherence_threshold


################################################################################
# 🧪 2. Класс CriticDataset — Dataset для обучения критика
################################################################################

class CriticDataset(Dataset):
    """
    Dataset для обучения critic: пары (токены, метка),
    где метка 1 — текст осмысленный, 0 — нет.
    """

    def __init__(self, sequences: List[List[int]], labels: List[int]):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx) -> Tuple[torch.Tensor, torch.Tensor]:
        return torch.tensor(self.sequences[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.float32)


################################################################################
# 🧪 3. Функция train_critic — обучение critic на размеченном корпусе
################################################################################

def train_critic(model: TorchCritic, dataset: CriticDataset, epochs: int = 5, lr: float = 1e-3):
    """
    Обучает critic по переданному датасету.
    """
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_batch)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.BCELoss()
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        for batch_x, batch_y in dataloader:
            probs = model(batch_x)
            loss = loss_fn(probs, batch_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}: loss = {total_loss/len(dataloader):.4f}")

################################################################################
# 🔧 4. collate_fn — функция для паддинга батчей
################################################################################

def collate_batch(batch: List[Tuple[torch.Tensor, torch.Tensor]]) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Паддинг батча до одинаковой длины по токенам.
    """
    sequences, labels = zip(*batch)
    padded = nn.utils.rnn.pad_sequence(sequences, batch_first=True, padding_value=0)
    return padded, torch.tensor(labels, dtype=torch.float32)


In [2]:
# critic.py — модуль для оценки "здравости" сгенерированного текста с помощью PyTorch

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from typing import List, Tuple
import random

################################################################################
# 🧠 1. Класс TorchCritic — модель для оценки осмысленности текста
################################################################################

class TorchCritic(nn.Module):
    """
    Простая модель-критик: Embedding → BiLSTM → Классификатор.
    Возвращает вероятность того, что текст "осмысленный".
    """

    def __init__(self, vocab_size: int, embed_dim: int = 64, hidden_dim: int = 128, coherence_threshold: float = 0.7):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)  # Преобразуем токены в векторы
        self.encoder = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.classifier = nn.Linear(2 * hidden_dim, 1)  # Предсказываем "осмысленность"
        self.coherence_threshold = coherence_threshold  # Порог, ниже которого отклоняется результат

    def forward(self, input_ids: torch.Tensor) -> torch.Tensor:
        """
        input_ids: (batch_size, seq_len) — батч токенов
        output: (batch_size,) — вероятность "осмысленности" для каждого примера
        """
        embedded = self.embedding(input_ids)                 # → (B, T, D)
        _, (hidden, _) = self.encoder(embedded)              # hidden: (2, B, H)
        concat_hidden = torch.cat((hidden[0], hidden[1]), dim=1)  # → (B, 2H)
        logits = self.classifier(concat_hidden)              # → (B, 1)
        probs = torch.sigmoid(logits)                        # → [0, 1]
        return probs.squeeze(1)                              # → (B,)

    def is_acceptable(self, input_ids: torch.Tensor) -> bool:
        """
        Возвращает True, если текст прошёл по порогу осмысленности.
        input_ids: (seq_len,) — одиночный пример
        """
        self.eval()
        with torch.no_grad():
            prob = self(input_ids.unsqueeze(0))  # → (1,)
            return prob.item() >= self.coherence_threshold


################################################################################
# 🧪 2. Класс CriticDataset — Dataset для обучения критика
################################################################################

class CriticDataset(Dataset):
    """
    Dataset для обучения critic: пары (токены, метка),
    где метка 1 — текст осмысленный, 0 — нет.
    """

    def __init__(self, sequences: List[List[int]], labels: List[int]):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx) -> Tuple[torch.Tensor, torch.Tensor]:
        return torch.tensor(self.sequences[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.float32)


################################################################################
# 🧪 3. Функция train_critic — обучение critic на размеченном корпусе
################################################################################

def train_critic(model: TorchCritic, dataset: CriticDataset, epochs: int = 5, lr: float = 1e-3):
    """
    Обучает critic по переданному датасету.
    """
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_batch)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.BCELoss()
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        for batch_x, batch_y in dataloader:
            probs = model(batch_x)
            loss = loss_fn(probs, batch_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}: loss = {total_loss/len(dataloader):.4f}")


################################################################################
# 🔧 4. collate_fn — функция для паддинга батчей
################################################################################

def collate_batch(batch: List[Tuple[torch.Tensor, torch.Tensor]]) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Паддинг батча до одинаковой длины по токенам.
    """
    sequences, labels = zip(*batch)
    padded = nn.utils.rnn.pad_sequence(sequences, batch_first=True, padding_value=0)
    return padded, torch.tensor(labels, dtype=torch.float32)


################################################################################
# 🧪 5. Пример генерации фейковых тренировочных данных и обучения
################################################################################

def make_dummy_training_data(vocab_size: int, num_samples: int = 100) -> Tuple[List[List[int]], List[int]]:
    """
    Генерирует фейковые примеры: осмысленные и бессмысленные (рандом и шум).
    """
    data, labels = [], []
    for _ in range(num_samples):
        length = random.randint(5, 20)
        # "Хороший" пример: плавные, возрастающие значения
        good = sorted(random.sample(range(10, vocab_size // 2), length))
        data.append(good)
        labels.append(1)

        # "Плохой" пример: шум, повторы
        bad = [random.randint(0, 5) for _ in range(length)]
        data.append(bad)
        labels.append(0)
    return data, labels

if __name__ == "__main__":
    vocab = 1000
    model = TorchCritic(vocab_size=vocab)
    sequences, labels = make_dummy_training_data(vocab_size=vocab, num_samples=200)
    dataset = CriticDataset(sequences, labels)
    train_critic(model, dataset, epochs=5)

    # Проверка
    test = torch.tensor(sequences[0])
    print("Acceptable?", model.is_acceptable(test))


Epoch 1: loss = 0.2659
Epoch 2: loss = 0.0231
Epoch 3: loss = 0.0060
Epoch 4: loss = 0.0018
Epoch 5: loss = 0.0009
Acceptable? True
